In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
import shutil
from functions import *

In [ ]:
def rotinaAlteracaoPrecos2(nome_relatorio, cod_grupo=None, categoria=None, laboratorio=None, excluir_categoria=False, excluir_laboratorio=False):
    extractEanCodShopfarma(nome_relatorio, 'cadastro', cod_grupo, categoria, laboratorio, excluir_categoria, excluir_laboratorio)
    df_cadastro = GeraDataframe(f'{nome_relatorio}_dados_cadastro.xlsx', 'cadastro')
    extractEanCodShopfarma(nome_relatorio, 'valores', cod_grupo, categoria, laboratorio, excluir_categoria, excluir_laboratorio)
    df_valores = GeraDataframe(f'{nome_relatorio}_dados_valores.xlsx', 'valores')
    df = pd.merge(df_cadastro, df_valores, on='codshopfarma')
    df.drop(['produto_x'], axis=1, inplace=True)
    df.rename({'produto_y': 'produto'}, axis=1, inplace=True)
    df['ean'] = (df['ean'].astype(str)).str.split('.').str[0]
    df['preco_venda'] = df['preco_venda'].str.replace('.', '').str.replace(',', '.').astype(float)
    df['preco_compra'] = df['preco_compra'].str.replace('.', '').str.replace(',', '.').astype(float)
    df['margem'] = (df['preco_venda']/df['preco_compra']-1)*100
    df['preco_compra'] = np.where(df['preco_compra'] < 0.5, np.NAN, df['preco_compra'])
    df['preco_venda'] = np.where(df['preco_venda'] < 10, np.NAN, df['preco_venda'])
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_excel(f'{nome_relatorio}.xls', index=False)
    return df

In [ ]:
df_perfumaria = rotinaAlteracaoPrecos2(nome_relatorio='perfumaria', cod_grupo='8000')
df_perfumaria

In [ ]:
df_perfumaria_precos = extrair_dados_concorrentes(df_perfumaria.head(20), nome_arquivo='perfumaria')

In [ ]:
import re

In [ ]:
def extrai_dados_saojoao(ean):
    url_principal = f'https://www.saojoaofarmacias.com.br/{ean}?_q={ean}'
    headers = {'user-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    site = requests.get(url_principal, headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    lk = soup.find_all(string=re.compile('Dorflex'))
    print(lk)
    #link_interno_saojoao = "https://www.saojoaofarmacias.com.br" + f"{lk}"
    #site_saojoao = requests.get(link_interno_saojoao, headers=headers)
    #soup_site_saojoao = BeautifulSoup(site_saojoao.content, 'html.parser')
    #cod_saojoao = ''
    #preco_saojoao = soup_site_saojoao.find_all('span', class_='sjdigital-custom-apps-0-x-currencyContainer')[1].get_text()
    #preco_saojoao = float(preco_saojoao.replace('R$','').replace('.','').replace(',','.'))
    #lst_saojoao = [link_interno_saojoao, cod_saojoao, preco_saojoao]
    #return lst_saojoao

In [ ]:
sao_joao = extrai_dados_saojoao('7891058022136')
sao_joao

df_genericos = rotinaAlteracaoPrecos(nome_relatorio='genericos', cod_grupo='4000')
df_genericos_precos = extrair_dados_concorrentes(df_genericos, nome_arquivo='genericos')

df_genericos_precos = pd.read_excel('precos_genericos.xlsx')
df_genericos_precos.head()

df_genericos_precos['desc_cliente'] = np.where(
    df_genericos_precos['desconto_pmedio'] > 40, 40,
    np.where(df_genericos_precos['desconto_pmedio'] < 10, 10, df_genericos_precos['desconto_pmedio'])
)

df_genericos_precos['desc_max_cliente'] = np.where(
    df_genericos_precos['desconto_pminimo'] > 70, 70,
    np.where(df_genericos_precos['desconto_pminimo'] < 20, 20, df_genericos_precos['desconto_pminimo'])
)

df_genericos_precos

df_genericos_precos.to_excel('precos_genericos.xlsx', index=False)

cod = '7896181924081'

extrai_dados_preco_popular(cod)

extrai_dados_saojoao(cod)

In [ ]:
def inclui_preco_cliente(df, filial=None):
    driver = webdriver.Chrome()#options=options)
    driver.get(f"http://179.184.16.200:4647/sgfpod1/Cad_0218.pod")
    sleep(1)
    driver.find_element(By.NAME, 'id_cod_usuario').send_keys(login_trier)
    sleep(1)
    driver.find_element(By.NAME, 'nom_senha').send_keys(password_trier)
    sleep(1)
    driver.find_element(By.NAME, 'login').click()
    sleep(1)
    if filial == None:
            pass
    else:
        driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(filial)
        driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(Keys.RETURN)
        sleep(1)
    driver.find_element(By.NAME, 'codTipoClienteEntrada').click()
    sleep(3)
    driver.find_element(By.NAME, 'codTipoClienteEntrada').send_keys('3')
    driver.find_element(By.NAME, 'codTipoClienteEntrada').send_keys(Keys.RETURN)
    driver.find_element(By.ID, 'produtoLabel').click()
    sleep(1)
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        try:
            driver.find_element(By.NAME, 'codTipoEntrada').send_keys(row['codshopfarma'])
            driver.find_element(By.NAME, 'codTipoEntrada').send_keys(Keys.RETURN)
            sleep(1)
            desconto_cliente = str(int(row['desc_cliente'])).replace('.',',')
            driver.find_element(By.NAME, 'prc_descto').send_keys(desconto_cliente)
            desconto_max_cliente = str(int(row['desc_max_cliente'])).replace('.',',')
            driver.find_element(By.NAME, 'prc_desctomaximo').send_keys(desconto_max_cliente)
            sleep(1)
            driver.find_element(By.NAME, 'salvar').click()
            sleep(3)
        except:
            try:
                driver.find_element(By.ID, 'bt_yes').click()
                sleep(3)
            except:
                driver.refresh()
                sleep(3)
                if filial == None:
                    driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(Keys.RETURN)
                else:
                    driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(filial)
                    driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(Keys.RETURN)
                    sleep(1)
                driver.find_element(By.NAME, 'codTipoClienteEntrada').click()
                sleep(3)
                driver.find_element(By.NAME, 'codTipoClienteEntrada').send_keys('3')
                driver.find_element(By.NAME, 'codTipoClienteEntrada').send_keys(Keys.RETURN)
                
                sleep(2)
    sleep(10)
        

In [ ]:
df = df_genericos_precos[1268:]
df

In [ ]:
df1 = df[1258:]

In [ ]:
df1

In [ ]:
inclui_preco_cliente(df1, filial='4')